In [2]:
from time import time_ns
from datetime import datetime , timedelta
import os
from pymongo import MongoClient
from pandas import DataFrame
from google.cloud import bigquery
import pandas as pd
from datetime import date

In [4]:
## resgitamos en las variables de entorno la clave de acceso a GCP
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "C:/Users/vik_c/Downloads/zeta-medley-405005-140cc1571703.json"

In [6]:
client = bigquery.Client(project='zeta-medley-405005')
    
sql = """
    SELECT *
    FROM `zeta-medley-405005.dep_raw.order_items`
"""

m_order_items_df = client.query(sql).to_dataframe()
        

In [7]:
sql_2 = """
SELECT *
FROM `zeta-medley-405005.dep_raw.orders`
"""
    
m_orders_df = client.query(sql_2).to_dataframe()

In [8]:
m_orders_df.count()

_id                  68883
order_id             68883
order_date           68883
order_customer_id    68883
order_status         68883
dtype: int64

In [9]:
m_order_items_df.count()

_id                         172198
order_date                  172198
order_item_id               172198
order_item_order_id         172198
order_item_product_id       172198
order_item_quantity         172198
order_item_subtotal         172198
order_item_product_price    172198
dtype: int64

In [10]:
df_join = m_orders_df.merge(m_order_items_df, left_on='order_id', right_on='order_item_order_id', how='inner')

In [11]:
df_join

,_id_x,order_id,order_date_x,order_customer_id,order_status,_id_y,order_date_y,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,65bf04a1bcc38f02f5ef4988,23969,2023-12-21,9970,CLOSED,65bf04d6bcc38f02f5f0e360,2023-12-21,60005,23969,1004,1,399.98,399.98
1,65bf04a1bcc38f02f5ef4995,23982,2023-12-21,4232,CLOSED,65bf04d6bcc38f02f5f0e380,2023-12-21,60037,23982,502,4,200.00,50.00
2,65bf04a1bcc38f02f5ef4995,23982,2023-12-21,4232,CLOSED,65bf04d6bcc38f02f5f0e381,2023-12-21,60038,23982,502,3,150.00,50.00
3,65bf04a1bcc38f02f5ef499f,23992,2023-12-21,11240,CLOSED,65bf04d6bcc38f02f5f0e389,2023-12-21,60046,23992,1014,4,199.92,49.98
4,65bf04a1bcc38f02f5ef499f,23992,2023-12-21,11240,CLOSED,65bf04d6bcc38f02f5f0e38a,2023-12-21,60047,23992,1014,4,199.92,49.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172193,65bf04a2bcc38f02f5efdc8f,61608,2023-12-20,2828,SUSPECTED_FRAUD,65bf04d6bcc38f02f5f252ba,2023-12-20,154047,61608,365,1,59.99,59.99
172194,65bf04a2bcc38f02f5efdc8f,61608,2023-12-20,2828,SUSPECTED_FRAUD,65bf04d6bcc38f02f5f252bb,2023-12-20,154048,61608,403,1,129.99,129.99
172195,65bf04a2bcc38f02f5efdc92,61611,2023-12-20,124,SUSPECTED_FRAUD,65bf04d6bcc38f02f5f252c1,2023-12-20,154054,61611,35,1,159.99,159.99
172196,65bf04a2bcc38f02f5efdc92,61611,2023-12-20,124,SUSPECTED_FRAUD,65bf04d6bcc38f02f5f252c2,2023-12-20,154055,61611,1004,1,399.98,399.98


In [12]:
df_master=df_join[[ 'order_id', 'order_date_x', 'order_customer_id',
   'order_status',  'order_item_id',
   'order_item_order_id', 'order_item_product_id', 'order_item_quantity',
   'order_item_subtotal', 'order_item_product_price']]

In [13]:
df_master=df_master.rename(columns={"order_date_x":"order_date"})

In [26]:
def get_group_status(text):
    text = str(text)
    if text =='CLOSED':
        d='END'
    elif text =='COMPLETE':
        d='END'
    else :
        d='TRANSIT'
    return d




In [17]:
df_master.count()

order_id                    172198
order_date                  172198
order_customer_id           172198
order_status                172198
order_item_id               172198
order_item_order_id         172198
order_item_product_id       172198
order_item_quantity         172198
order_item_subtotal         172198
order_item_product_price    172198
order_status_group          172198
dtype: int64

In [18]:
df_master['order_status_group']  = df_master['order_status'].map(get_group_status)

In [19]:
df_master['order_date'] = df_master['order_date'].astype(str)

In [20]:
df_master['order_date'] = pd.to_datetime(df_master['order_date'], format='%Y-%m-%d').dt.date

In [23]:
headers_filter = {
    'tipocambio': ['fecha' ,'compra','venta','nan']
    }
    
dwn_url_tipocambio= 'https://www.sunat.gob.pe/a/txt/tipoCambio.txt'
df = pd.read_csv(dwn_url_tipocambio, names=headers_filter['tipocambio'],sep='|')
list_t= df.values.tolist()

var1 = list_t[0][1]

In [30]:
def get_national_currency(amount):
    return amount * var1

In [32]:
df_master['order_item_subtotal_mn']  = df_master['order_item_subtotal'].map(get_national_currency)

In [ ]:
df_master

In [ ]:
master_rows=len(df_master)
    print(f" Se obtuvo  {master_rows}  Filas")

    df_master_rows=len(df_master)
    
    if df_master_rows>0 :
        client = bigquery.Client()
    
        table_id =  "zeta-medley-405005.dep_raw.master_order"
        job_config = bigquery.LoadJobConfig(
            schema=[
                bigquery.SchemaField("order_id", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("order_date", bigquery.enums.SqlTypeNames.DATE),
                bigquery.SchemaField("order_customer_id", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("order_status", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("order_item_id", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("order_item_order_id", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("order_item_product_id", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("order_item_quantity", bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField("order_item_subtotal", bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField("order_item_product_price", bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField("order_status_group", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("order_item_subtotal_mn", bigquery.enums.SqlTypeNames.FLOAT),
            ],
            write_disposition="WRITE_TRUNCATE",
        )
    
    
        job = client.load_table_from_dataframe(
            df_master, table_id, job_config=job_config
        )  
        job.result()  # Wait for the job to complete.
    
        table = client.get_table(table_id)  # Make an API request.
        print(
            "Loaded {} rows and {} columns to {}".format(
                table.num_rows, len(table.schema), table_id
            )
        )
    else : 
        print('alerta no hay registros en la tabla master_order')